### Word Group In Youtube

In [83]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [84]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [85]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:1000
file_ext:6
unique_word_ratio:80
include_wordgroup_ratio:80
word_usage_min:10
word_usage_max:100


In [86]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [87]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [88]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [89]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [90]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [91]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [92]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
4995,sworn,22691
4996,fame,22691
4997,zombie,22681
4998,photographer,22673


In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

4994

#### Youtube Sentence Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:29.416,00:00:33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,00:00:33.810,00:00:36.876,shivering cold and snow,tPffr9WPl9Y
2,00:00:37.150,00:00:39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,00:00:39.257,00:00:43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,00:00:43.796,00:00:50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...
5742563,00:02:59.400,00:03:03.030,wounded and,78VHQeebbWM
5742564,00:03:03.030,00:03:10.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,00:03:10.590,00:03:13.159,facebook on december,78VHQeebbWM
5742566,00:03:13.159,00:03:16.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [ ]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,50.0,jingle,"way, all, bell, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,75.0,shivering,"and, snow, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,90.0,festive,"season, you, wish, do, what, for, else, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,64.3,"decorated, colours, edinburgh","has, the, lights, with, and, christmas, of, he..."
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,82.3,"edinburgh, explore, celebrated","here, so, we, today, how, will, being, places,..."
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,100.0,,"and, wounded"
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,78.6,"ukraine, operational","been, has, the, forces, information, staff, sh..."
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,66.6,facebook,"on, december"
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,88.9,comments,"express, can, thoughts, the, section, your, yo..."


In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,50.0,jingle,"way, all, bell, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,75.0,shivering,"and, snow, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,90.0,festive,"season, you, wish, do, what, for, else, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,64.3,"decorated, colours, edinburgh","has, the, lights, with, and, christmas, of, he..."
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,82.3,"edinburgh, explore, celebrated","here, so, we, today, how, will, being, places,..."
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,100.0,NaN,"and, wounded"
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,78.6,"ukraine, operational","been, has, the, forces, information, staff, sh..."
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,66.6,facebook,"on, december"
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,88.9,comments,"express, can, thoughts, the, section, your, yo..."


In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,50.0,jingle,"way, all, bell, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,75.0,shivering,"and, snow, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,90.0,festive,"season, you, wish, do, what, for, else, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,64.3,"decorated, colours, edinburgh","has, the, lights, with, and, christmas, of, he..."
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,82.3,"edinburgh, explore, celebrated","here, so, we, today, how, will, being, places,..."
...,...,...,...,...,...,...,...,...
5690257,5742563,179.400,183.030,wounded and,78VHQeebbWM,100.0,NaN,"and, wounded"
5690258,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,78.6,"ukraine, operational","been, has, the, forces, information, staff, sh..."
5690259,5742565,190.590,193.159,facebook on december,78VHQeebbWM,66.6,facebook,"on, december"
5690260,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,88.9,comments,"express, can, thoughts, the, section, your, yo..."


In [ ]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-65-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,90.0,festive,"season, you, wish, do, what, for, else, this, in",what else do you wish for in this
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,82.3,"edinburgh, explore, celebrated","here, so, we, today, how, will, being, places,...",and experience how christmas is being
5,5,50.504,51.928,so lets go with me,tPffr9WPl9Y,100.0,NaN,"so, go, me, with, lets",so lets go with me
6,6,54.001,61.359,with the beginning of winter in november chris...,tPffr9WPl9Y,76.9,"preparations, edinburgh","winter, november, the, started, with, beginnin...",with the beginning of winter in november chris...
7,7,62.231,66.363,people of edinburgh wait for these few days th...,tPffr9WPl9Y,90.9,edinburgh,"these, throughout, the, wait, year, few, days,...",wait for these few days throughout the year
...,...,...,...,...,...,...,...,...,...
5690255,5742561,167.800,173.610,it is reported that as a result of artillery f...,78VHQeebbWM,81.2,"artillery, concentration","damage, result, it, fire, to, reported, as, ru...",it is reported that as a result of
5690256,5742562,173.610,179.400,the city of starobilsk luhansk region the russ...,78VHQeebbWM,72.7,"starobilsk, luhansk","the, region, russian, troops, of, lost, about,...",region the russian troops lost about
5690258,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,78.6,"ukraine, operational","been, has, the, forces, information, staff, sh...",general staff of the armed forces of
5690260,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,88.9,comments,"express, can, thoughts, the, section, your, yo...",you can express your thoughts in the


In [ ]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,what else do you wish for in this,37.150000,38.460417,what else do you wish for in this festive season,tPffr9WPl9Y
1,and experience how christmas is being,46.945000,49.219278,so today we will explore different places in e...,tPffr9WPl9Y
2,so lets go with me,50.504000,51.928000,so lets go with me,tPffr9WPl9Y
3,with the beginning of winter in november chris...,54.001000,58.265295,with the beginning of winter in november chris...,tPffr9WPl9Y
4,wait for these few days throughout the year,63.477159,66.363000,people of edinburgh wait for these few days th...,tPffr9WPl9Y
...,...,...,...,...,...
3576280,it is reported that as a result of,167.800000,170.164535,it is reported that as a result of artillery f...,78VHQeebbWM
3576281,region the russian troops lost about,176.202537,179.400000,the city of starobilsk luhansk region the russ...,78VHQeebbWM
3576282,general staff of the armed forces of,183.030000,186.245172,general staff of the armed forces of ukraine o...,78VHQeebbWM
3576283,you can express your thoughts in the,193.159000,195.679189,you can express your thoughts in the comments ...,78VHQeebbWM


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,what else do you wish for in this,36.850000,38.760417,what else do you wish for in this festive season,tPffr9WPl9Y
1,and experience how christmas is being,46.645000,49.519278,so today we will explore different places in e...,tPffr9WPl9Y
2,so lets go with me,50.204000,52.228000,so lets go with me,tPffr9WPl9Y
3,with the beginning of winter in november chris...,53.701000,58.565295,with the beginning of winter in november chris...,tPffr9WPl9Y
4,wait for these few days throughout the year,63.177159,66.663000,people of edinburgh wait for these few days th...,tPffr9WPl9Y
...,...,...,...,...,...
3576280,it is reported that as a result of,167.500000,170.464535,it is reported that as a result of artillery f...,78VHQeebbWM
3576281,region the russian troops lost about,175.902537,179.700000,the city of starobilsk luhansk region the russ...,78VHQeebbWM
3576282,general staff of the armed forces of,182.730000,186.545172,general staff of the armed forces of ukraine o...,78VHQeebbWM
3576283,you can express your thoughts in the,192.859000,195.979189,you can express your thoughts in the comments ...,78VHQeebbWM


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,what else do you wish for in this,37,39,what else do you wish for in this festive season,tPffr9WPl9Y
1,and experience how christmas is being,47,50,so today we will explore different places in e...,tPffr9WPl9Y
2,so lets go with me,50,52,so lets go with me,tPffr9WPl9Y
3,with the beginning of winter in november chris...,54,59,with the beginning of winter in november chris...,tPffr9WPl9Y
4,wait for these few days throughout the year,63,67,people of edinburgh wait for these few days th...,tPffr9WPl9Y
...,...,...,...,...,...
3576280,it is reported that as a result of,168,170,it is reported that as a result of artillery f...,78VHQeebbWM
3576281,region the russian troops lost about,176,180,the city of starobilsk luhansk region the russ...,78VHQeebbWM
3576282,general staff of the armed forces of,183,187,general staff of the armed forces of ukraine o...,78VHQeebbWM
3576283,you can express your thoughts in the,193,196,you can express your thoughts in the comments ...,78VHQeebbWM


In [ ]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,and and i guess that the when we talk about be...,1038,1065,jung kian ng and and i guess that the when we ...,dIvJL4OKiWQ
1,um i think like going with me said as well jus...,3258,3282,kailyn xu um i think like going with me said a...,iJkfTPN8MS0
2,um because that was like a big a big thing in ...,1135,1155,sydnee yu um because that was like a big a big...,iJkfTPN8MS0
3,and it was fully remote um but what was really...,1018,1034,kailyn xu awesome um i think im gon na start w...,iJkfTPN8MS0
4,how do you keep going after the nose because i...,857,871,maria otero sheher how do you keep going after...,jO_jH5b443E
...,...,...,...,...,...
3576280,e i e i o,194,197,e i e i o,yGC9K3yEO9Y
3576281,e i e i o,199,202,e i e i o,yGC9K3yEO9Y
3576282,e i e i o,209,212,e i e i o,yGC9K3yEO9Y
3576283,e i e i o,219,222,e i e i o,yGC9K3yEO9Y


In [ ]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,and and i guess that the when we talk about be...,1038,1065,jung kian ng and and i guess that the when we ...,dIvJL4OKiWQ,60
1,um i think like going with me said as well jus...,3258,3282,kailyn xu um i think like going with me said a...,iJkfTPN8MS0,65
2,um because that was like a big a big thing in ...,1135,1155,sydnee yu um because that was like a big a big...,iJkfTPN8MS0,70
3,and it was fully remote um but what was really...,1018,1034,kailyn xu awesome um i think im gon na start w...,iJkfTPN8MS0,71
4,how do you keep going after the nose because i...,857,871,maria otero sheher how do you keep going after...,jO_jH5b443E,61
...,...,...,...,...,...,...
3576280,e i e i o,194,197,e i e i o,yGC9K3yEO9Y,60
3576281,e i e i o,199,202,e i e i o,yGC9K3yEO9Y,60
3576282,e i e i o,209,212,e i e i o,yGC9K3yEO9Y,60
3576283,e i e i o,219,222,e i e i o,yGC9K3yEO9Y,60


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-71-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-71-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,without having um as much structure or as much...,2093,2111,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0
1,lets talk about your action plans now and i an...,1870,1886,lets talk about your action plans now and i an...,nDOjm1UKsNM
2,here in as an example of hours of operation ri...,933,951,here in as an example of hours of operation ri...,nDOjm1UKsNM
3,norman jones remember the kind of big picture ...,2320,2335,norman jones remember the kind of big picture ...,b9pudZegYgQ
4,de rose we appreciate you being here im glad i...,2799,2809,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs
...,...,...,...,...,...
3339085,l m n o p,208,213,l m n o p,wKC7h0uBFV4
3339086,t u p i d,111,112,s t u p i d !,3KsbG3WCQuk
3339087,b a y o u,6,7,bayou mmmm mmmmm b a y o u,k6rleNtZH3U
3339088,l m n o p,650,655,l m n o p,6aYMOPerAz8


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-72-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,without having um as much structure or as much...,2093,2111,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0,85
1,lets talk about your action plans now and i an...,1870,1886,lets talk about your action plans now and i an...,nDOjm1UKsNM,100
2,here in as an example of hours of operation ri...,933,951,here in as an example of hours of operation ri...,nDOjm1UKsNM,100
3,norman jones remember the kind of big picture ...,2320,2335,norman jones remember the kind of big picture ...,b9pudZegYgQ,89
4,de rose we appreciate you being here im glad i...,2799,2809,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,86
...,...,...,...,...,...,...
3339085,l m n o p,208,213,l m n o p,wKC7h0uBFV4,100
3339086,t u p i d,111,112,s t u p i d !,3KsbG3WCQuk,69
3339087,b a y o u,6,7,bayou mmmm mmmmm b a y o u,k6rleNtZH3U,34
3339088,l m n o p,650,655,l m n o p,6aYMOPerAz8,100


In [ ]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-73-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,without having um as much structure or as much...,2093,2111,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0
1,lets talk about your action plans now and i an...,1870,1886,lets talk about your action plans now and i an...,nDOjm1UKsNM
2,here in as an example of hours of operation ri...,933,951,here in as an example of hours of operation ri...,nDOjm1UKsNM
3,norman jones remember the kind of big picture ...,2320,2335,norman jones remember the kind of big picture ...,b9pudZegYgQ
4,de rose we appreciate you being here im glad i...,2799,2809,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs
...,...,...,...,...,...
1526485,d e f g h,324,327,d e f g h,wKC7h0uBFV4
1526486,l m n o p,280,285,l m n o p,wKC7h0uBFV4
1526487,l m n o p,208,213,l m n o p,wKC7h0uBFV4
1526488,l m n o p,650,655,l m n o p,6aYMOPerAz8


In [ ]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,something extraordinary something which makes us,zyOCtVxkRy4
1,why does she choose television specifically,zyOCtVxkRy4
2,pretty good clear time yesterday actually um l...,zyEMeStyuB8
3,do ya ever wan na get married,zxcNJO9nlsw
4,if they complete an impossible job by christma...,zxKBVSiuENw
...,...,...
3614,rather than drawing everything perfectly,01J8qKjcp0M
3615,then everything probably flies off,01J8qKjcp0M
3616,see okay seven seconds remaining,01J8qKjcp0M
3617,lets see does anybody recognize themselves,01J8qKjcp0M


In [ ]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,without having um as much structure or as much...,2093,2111,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0
1,lets talk about your action plans now and i an...,1870,1886,lets talk about your action plans now and i an...,nDOjm1UKsNM
2,here in as an example of hours of operation ri...,933,951,here in as an example of hours of operation ri...,nDOjm1UKsNM
3,norman jones remember the kind of big picture ...,2320,2335,norman jones remember the kind of big picture ...,b9pudZegYgQ
4,de rose we appreciate you being here im glad i...,2799,2809,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs
...,...,...,...,...,...
3545,h u log d plus,2262,2273,h u log d plus,EHvq5jvgV0o
3546,l m n o p,110,112,l m n o p,EagLlPDC3Tw
3547,d e f g h,324,327,d e f g h,wKC7h0uBFV4
3548,l m n o p,280,285,l m n o p,wKC7h0uBFV4


In [ ]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,without having um as much structure or as much...,2093,2111,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0,https://www.youtube.com/watch?v=iJkfTPN8MS0&t=...
1,lets talk about your action plans now and i an...,1870,1886,lets talk about your action plans now and i an...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
2,here in as an example of hours of operation ri...,933,951,here in as an example of hours of operation ri...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
3,norman jones remember the kind of big picture ...,2320,2335,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
4,de rose we appreciate you being here im glad i...,2799,2809,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
...,...,...,...,...,...,...
3545,h u log d plus,2262,2273,h u log d plus,EHvq5jvgV0o,https://www.youtube.com/watch?v=EHvq5jvgV0o&t=...
3546,l m n o p,110,112,l m n o p,EagLlPDC3Tw,https://www.youtube.com/watch?v=EagLlPDC3Tw&t=...
3547,d e f g h,324,327,d e f g h,wKC7h0uBFV4,https://www.youtube.com/watch?v=wKC7h0uBFV4&t=...
3548,l m n o p,280,285,l m n o p,wKC7h0uBFV4,https://www.youtube.com/watch?v=wKC7h0uBFV4&t=...


In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['English_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx']

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass